Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.ru/news
Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

Сложить все новости в БД

In [ ]:
#conda install -c conda-forge pendulum

In [1]:
from lxml import html
import pendulum
import requests
from pprint import pprint
from pymongo import MongoClient
import pandas as pd

In [2]:
class news():
    
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.101 YaBrowser/20.7.0.1230 Yowser/2.5 Safari/537.36',
        'Accept': '*/*'}

    def get_news_mail(self):
        news=[]
        main_link = 'https://news.mail.ru'
        response = requests.get(main_link, headers = self.header)
        dom = html.fromstring(response.text)
        links=dom.xpath("//div[@class='daynews__item']/a/@href")
        links+=dom.xpath("//a[@class='list__text']/@href")
        for link in links:
            new={}
            new['link']=main_link+link
            if 'http' in link:
                response = requests.get(link, headers=self.header)
            else:
                response = requests.get(main_link+link, headers = self.header)
            dom = html.fromstring(response.text)
            new['source'] =dom.xpath("//span[@class='breadcrumbs__item']//span[@class='link__text']/text()")[0]
            new['name'] =dom.xpath("//h1/text()")[0]
            new['date'] = dom.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0][:10]
            news.append(new)
        return news
    
    def get_news_yandex(self):
        news = []
        main_link = 'https://yandex.ru'
        response = requests.get(main_link+'/news', headers=self.header)
        dom = html.fromstring(response.text)
        links = dom.xpath("//td[@class='stories-set__item']")
        for link in links:
            new = {}
            new['link'] = main_link + link.xpath(".//h2/a/@href")[0]
            new['name'] = link.xpath(".//h2/a/text()")[0]
            text = link.xpath(".//div[@class='story__date']/text()")[0]
            source = ''
            lst = text.split()
            for i in lst:
                if i == 'вчера' or ':' in i:
                    break
                else:
                    source += i + " "
            source = source.strip()
            new['source'] = source
            if 'вчера' in text:
                new['date'] = pendulum.yesterday('Europe/Moscow').format('DD.MM.YYYY')
            else:
                new['date'] = pendulum.today('Europe/Moscow').format('DD.MM.YYYY')
            news.append(new)
        return news
    
    def get_news_lenta(self):
        news = []
        main_link = 'https://lenta.ru/'
        response = requests.get(main_link , headers=self.header)
        dom = html.fromstring(response.text)
        links = dom.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item']")
        for link in links:
            new = {}
            new['link'] = main_link + link.xpath("./a/@href")[0]
            new['name'] = link.xpath("./a/text()")[0].replace('\xa0',"")
            new['source'] = 'Lenta.ru'
            new['date'] = link.xpath(".//time/@title")[0]
            news.append(new)
        link = dom.xpath("//div[@class='first-item']")[0]
        new = {}
        new['link'] = main_link + link.xpath("./a/@href")[0]
        new['name'] = link.xpath("./h2/a/text()")[0].replace('\xa0'," ")
        new['source'] = 'Lenta.ru'
        new['date'] = link.xpath(".//time/@title")[0]
        news.append(new)
        return news

In [3]:
class insert_to_mongodb():
    def __init__(self, my_db,my_collecton, connect_str='localhost:27017'):
        self.connect_str=connect_str
        self.my_db=my_db
        self.my_collection=my_collecton
    def __connect(self):
        try:
            client = MongoClient(self.connect_str)
            db = client[self.my_db]
            collection = db[self.my_collection]
            return  collection
        except Exception as e:
            print("нет подключения" + str(e))

    def only_insert_all_data(self,my_data):
        cursor=self.__connect()
        cursor.insert_many(my_data)

    def add_new(self,my_data):
        cursor = self.__connect()
        for i in my_data:
            cursor.update_one({'link':i['link']},{'$set':i},upsert=True)

In [4]:
if __name__ == '__main__':
    news=news()
    load=insert_to_mongodb('news','yandex')
    load.add_new(news.get_news_yandex())
    pprint (news.get_news_yandex())

[{'date': '25.07.2020',
  'link': 'https://yandex.ru/news/story/Ismailov_nokautiroval_Aleksandra_Emelyanenko_na_turnire_ASA_107--498d2e24636ad00378240665e2edb1b3?lr=9&lang=ru&stid=oOq0GuG2ko7NqrJSQarN&persistent_id=107238318&rubric=index&from=index',
  'name': 'Исмаилов нокаутировал Александра Емельяненко на турнире АСА 107',
  'source': 'Газета.Ru'},
 {'date': '25.07.2020',
  'link': 'https://yandex.ru/news/story/Pri_preodolenii_Kerchenskogo_proliva_zatonula_bronemashina--d24db0b9b5f4a44e3807c3795a5f42ad?lr=9&lang=ru&stid=_3uqfxNNCKqdySAl_YvN&persistent_id=106853939&rubric=index&from=index',
  'name': 'При преодолении Керченского пролива затонула бронемашина',
  'source': 'Коммерсантъ'},
 {'date': '25.07.2020',
  'link': 'https://yandex.ru/news/story/V_Rossii_za_sutki_vyyavili_5871_zabolevshego_COVID-19--7d0a1dc3ea87cb79f5d70d08b107345a?lr=9&lang=ru&stid=CRxHCLJZ9Rof3os9N8zJ&persistent_id=107280617&rubric=index&from=index',
  'name': 'В России за сутки выявили 5871 заболевшего COVID-1

In [5]:
load = insert_to_mongodb('news', 'mail')
load.add_new(news.get_news_mail())
pprint(news.get_news_mail())

[{'date': '2020-07-25',
  'link': 'https://news.mail.ruhttps://sportmail.ru/news/wrestling/42713601/',
  'name': 'В Подмосковье найден мертвым пропавший борец и спортивный журналист '
          'Иваницкий',
  'source': 'ТАСС'},
 {'date': '2020-07-25',
  'link': 'https://news.mail.ru/economics/42712256/',
  'name': 'ЦБ опустил ставку до минимума. Что будет с вкладами, ипотекой и '
          'рублем',
  'source': 'РИА Новости'},
 {'date': '2020-07-25',
  'link': 'https://news.mail.ru/incident/42714479/',
  'name': 'Еще два дела завели из-за фото нацистов на сайте «Бессмертного '
          'полка»',
  'source': 'РИА Новости'},
 {'date': '2020-07-25',
  'link': 'https://news.mail.ruhttps://sportmail.ru/news/mma/42711294/',
  'name': 'Врачи диагностировали перелом носа у Александра Емельяненко',
  'source': 'ТАСС'},
 {'date': '2020-07-25',
  'link': 'https://news.mail.ru/incident/42714479/',
  'name': 'Еще два дела завели из-за фото нацистов на сайте «Бессмертного '
          'полка»',
  's

In [6]:
load = insert_to_mongodb('news', 'lenta')
load.add_new(news.get_news_lenta())
pprint(news.get_news_lenta())

[{'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/delphin/',
  'name': 'В крымском дельфинарии расчленили дельфина',
  'source': 'Lenta.ru'},
 {'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/kharitosha/',
  'name': 'Проигравшему нокаутом Емельяненко посоветовали уходить встриптиз',
  'source': 'Lenta.ru'},
 {'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/grad/',
  'name': 'Захвативший монастырь священник предупредил обоевой тревоге '
          '«Небесный град»',
  'source': 'Lenta.ru'},
 {'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/zhirinovskii/',
  'name': 'Жириновский пожелал поменять столицу Дальнего Востока',
  'source': 'Lenta.ru'},
 {'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/eto_moe/',
  'name': 'Якубович отказался уходить с «Поля чудес»',
  'source': 'Lenta.ru'},
 {'date': '25 июля 2020',
  'link': 'https://lenta.ru//news/2020/07/25/dela/',
  'name': 'СК возбудил 